Overfitted Model - Changes were made to optimizer, activation function and learning rate

In [16]:
# All the code lines in this cell are taken from Reference [4]

import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader


# train and validation data directory
train_data_dir = "images/images/train/"
validation_data_dir = "images/images/validation/"


# load the train and validation data
train_dataset = ImageFolder(train_data_dir,transform = transforms.Compose([
                transforms.ToTensor()
]))
validation_dataset = ImageFolder(validation_data_dir,transforms.Compose([
                transforms.ToTensor()
]))


# load the train and validation into batches.
train_dl = DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers = 4)

# Few changes (batch_size, shuffle = False) were made to val_dl object by referring to Reference[1]
val_dl = DataLoader(validation_dataset, batch_size = 128, shuffle = False, num_workers = 4)

# All the code lines in this cell are taken from Reference [1]

import torch.nn as nn # basic building block for neural networks
import torch.nn.functional as F # import convolution functions like Relu

class Net(nn.Module):
# Models a simple Convolutional Neural Network
	
    def __init__(self):
# initialize the network 
        super(Net, self).__init__() 

# The parameters of Conv2d and Linear layers are modified slightly from the original code.

# 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
    
# Max pooling over a (2, 2) window
        self.pool = nn.MaxPool2d(2, 2)
         
        self.fc1 = nn.Linear(64 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 7)

    def forward(self, x):
# the forward propagation algorithm
        x = self.pool(F.elu(self.conv1(x)))
        x = self.pool(F.elu(self.conv2(x)))

# The parameters of view are modified slightly from the original code.        
        x = x.view(-1, 64 * 10 * 10)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

# All the code lines in this cell are taken from Reference [1]

import torch.optim as optim # optimizer

criterion = nn.CrossEntropyLoss()

# 'lr' value is modified from the original code.
optimizer = optim.Adam(net.parameters(), lr = 0.001)

# The below code lines are taken from Reference [1]

# 'Number of epochs'is modified from the original code.
import time                       # code line referred from Reference [7]

train_start_time = time.time()    # code line referred from Reference [7]
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# The below code lines are taken from Reference [5]
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
       
        train_loss2=running_loss/len(train_dl)
        accu2=100.*correct/total
   
train_end_time = time.time()                      # code line referred from Reference [7]
t_time2 = train_end_time - train_start_time       # code line referred from Reference [7]
       
print("Training time : %.2fs"%(t_time2))          # code line referred from Reference [7]

print('\nLoss for train dataset : %.3f | Accuracy for train dataset : %.3f'%(train_loss2,accu2))

# The below code lines are taken from Reference [1]
# 'Number of epochs' is modified from the original code.

for epoch in range(10):
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_dl:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

# The below code lines are taken from Reference [5]
            test_loss2=running_loss/len(val_dl)
            test_accu2=100.*correct/total
            

print('\nLoss for validation dataset : %.3f | Accuracy for validation dataset : %.3f'%(test_loss2,test_accu2))

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6400, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=7, bias=True)
)
Training time : 990.74s

Loss for train dataset : 0.230 | Accuracy for train dataset : 92.127

Loss for validation dataset : 0.928 | Accuracy for validation dataset : 54.104
